# Ejemplo con Pandas de un Zoo

## Imports

In [ ]:
import numpy as np
import pandas as pd

## Leemos el CSV

In [ ]:
zoo = pd.read_csv('zoo.csv', delimiter = ',')

## Contamos los elementos de cada columna

In [ ]:
zoo.count()

## Contamos los elementos de la columna animal

In [ ]:
zoo[['animal']].count()

## Sumatorio del agua que necesitan todos los animales

In [ ]:
zoo.water_need.sum()

## Algunos calculos

In [ ]:
minimo_agua = zoo.water_need.min()
maximo_agua = zoo.water_need.max()
media_agua = zoo.water_need.mean()
mediana_agua = zoo.water_need.median()

print("Minimo de necesidad de agua:", minimo_agua)
print("Maximo de necesidad de agua:", maximo_agua)
print("Media de necesidad de agua:", media_agua)
print("Mediana de necesidad de agua:", mediana_agua)

## Localicemos a los animales de menor y mayor necesidad de agua

In [ ]:
zoo.loc[zoo["water_need"] == minimo_agua,["animal","water_need"]]

In [ ]:
zoo.loc[zoo["water_need"] == maximo_agua,["animal","water_need"]]

# Agrupación (grouping) en Pandas
<img src="./zoo_1.png"  />

## Calcular la media agrupado por animal

In [ ]:
# Nos calcula la media de TODAS las columnas
zoo.groupby('animal').mean()

In [ ]:
# Nos calcula la media de la columna water_need
zoo.groupby('animal')[['water_need']].mean()

In [ ]:
# Alternativa del anterior
zoo.groupby('animal').mean().water_need

# Unión (merge - join) en Pandas

## Cargamos el csv de zoo_eats.csv

In [ ]:
zoo_eats = pd.read_csv('zoo_eats.csv', delimiter = ',')
zoo_eats

## Unimos los dos dataframes

In [ ]:
zoo.merge(zoo_eats)

## Diferentes tipos de unión
Como se puede ver, el método de unión básico es bastante simple. Sin embargo, a veces se tiene que agregar algunos parámetros adicionales. En SQL, aprendimos que existen diferentes tipos de JOIN.
<img src="./zoo_2.png"  />

## OUTER JOIN
Cuando haces un INNER JOIN (que es el predeterminado tanto en SQL como en pandas), fusionas solo los valores que se encuentran en ambas tablas. Por otro lado, cuando realiza OUTER JOIN, fusiona todos los valores, incluso si puede encontrar algunos de ellos en solo una de las tablas.

Veamos un ejemplo concreto: ¿te diste cuenta de que no hay valor de león en zoo_eats? ¿O que no tenemos jirafas en el zoológico? Cuando hicimos la combinación anterior, de forma predeterminada, era una combinación INNER, por lo que filtró jirafas y leones de la tabla de resultados. Pero hay casos en los que queremos ver estos valores en nuestro resultado. Intentemos esto:

<img src="./zoo_3.png"  />

In [ ]:
zoo.merge(zoo_eats, how = 'outer')

## LEFT JOIN
¿Ver? Los leones regresaron, la jirafa regresó ... Lo único es que tenemos valores vacíos (NaN) en esas columnas donde no obtuvimos información de la otra tabla.

En mi opinión, en este caso concreto, tendría más sentido mantener a los leones en la tabla pero no a las jirafas… Con eso, podríamos ver todos los animales de nuestro zoológico y tendríamos tres categorías de alimentos: vegetales, carne y NaN. (que es básicamente "sin información"). Mantener la línea de jirafas sería engañoso e irrelevante, ya que de todos modos no tenemos jirafas en nuestro zoológico. ¡Es entonces cuando se vuelve útil fusionar con un parámetro how = 'left'!

<img src="./zoo_4.png"  />

In [ ]:
zoo.merge(zoo_eats, how = 'left')

## CLAVES POR DEFECTO
Por defecto, Pandas selecciona las claves (columnas) por las que va a realizar la combinación en ambas tablas. En este caso ha acertado porque ha seleccionado la columna animal tanto en la tabla de la izquierda como en la tabla de la derecha.

Pero si Pandas falla o queremos tener un mayor control sobre nuestro código, usaremos los parámetros left_on y right_on

In [ ]:
zoo.merge(zoo_eats, how = 'left', left_on = 'animal', right_on = 'animal')

## UNIÓN FINAL

In [ ]:
zoo_final = zoo.merge(zoo_eats, how = 'left')

In [ ]:
zoo_final = zoo.merge(zoo_eats, how = 'left').fillna('unknown')
zoo_final

## ORDENACIÓN

In [ ]:
zoo_final.sort_values('water_need') # Por water_need de menor a mayor

In [ ]:
zoo_final.sort_values(by = ['animal', 'water_need']) # Por animal y water_need

In [ ]:
zoo_final.sort_values(by = ['water_need'], ascending = False) # Por water_need de mayor a menor

In [ ]:
zoo_final.sort_values(by = ['water_need'], ascending = False).reset_index() # Por water_need de mayor a menor y reseteando indices

In [ ]:
# En el paso anterior, reseteo los indices pero se han mantenido los antiguos, vamos a borrarlos
zoo_final.sort_values(by = ['water_need'], ascending = False).reset_index(drop = True)

# ÚLTIMOS DETALLES

In [ ]:
# Vamos a quitar la columna de uniq_id, ya que no queremos que se exporte
zoo_final = zoo_final.drop(['uniq_id'], axis=1)
zoo_final

In [ ]:
# Creacion de una nueva columna
zoo_final["water_need_x2"] = zoo_final["water_need"]*2
zoo_final

# EXPORTACIÓN

## EXPORTACIÓN DIRECTA A CSV

In [ ]:
ruta_csv = r"./EXPORT_zoo_final.csv"
zoo_final.to_csv(ruta_csv, index = False, header=True, sep=",")

## EXPORTACIÓN DIRECTA A XLSX

In [ ]:
ruta_excel = r"./EXPORT_zoo_final.xlsx"
zoo_final.to_excel(ruta_excel, index = False, header=True)

## EXPORTACIÓN DIRECTA A JSON
Tenemos varios formatos de JSON:
* columns (por defecto)
* table
* values
* index
* records

In [ ]:
ruta_json = r"./EXPORT_zoo_final.json"
zoo_final.to_json(ruta_json, orient="columns")
# Comprobar con https://jsonformatter.org/json-viewer

## EXPORTACIÓN DIRECTA A HTML

In [ ]:
ruta_html = r"./EXPORT_zoo_final.html"
zoo_final.to_html(ruta_html)

## EXPORTACIÓN A CSV MEDIANTE TKINTER

In [ ]:
import tkinter as tk
from tkinter import filedialog

root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 300, bg = 'lightsteelblue2', relief = 'raised')
canvas1.pack()

def exportCSV ():
    global df
    
    export_file_path = filedialog.asksaveasfilename(defaultextension='.csv')
    zoo_final.to_csv (export_file_path, index = False, header=True)

saveAsButton_CSV = tk.Button(text='Export CSV', command=exportCSV, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 150, window=saveAsButton_CSV)

root.mainloop()

## EXPORTACIÓN A XLSX MEDIANTE TKINTER

In [ ]:
import tkinter as tk
from tkinter import filedialog

root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 300, bg = 'lightsteelblue2', relief = 'raised')
canvas1.pack()

def exportExcel ():
    global df
    
    export_file_path = filedialog.asksaveasfilename(defaultextension='.xlsx')
    zoo_final.to_excel (export_file_path, index = False, header=True)

saveAsButtonExcel = tk.Button(text='Export Excel', command=exportExcel, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 150, window=saveAsButtonExcel)

root.mainloop()

# GRÁFICOS

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Nos calcula la media de la columna water_need
agua_necesaria = zoo_final.groupby('animal')[['water_need']].sum()
agua_necesaria

In [ ]:
# Generar gráfico
grafico = agua_necesaria.plot(kind='bar', figsize=(8, 5), fontsize=26).get_figure()
plt.show()

In [ ]:
# Guardar gráfico
grafico.savefig("AguaNecesaria.jpg", bbox_inches='tight', dpi=800)

# EXPORTAR DATAFRAME CON IMAGEN A EXCEL

In [ ]:
# DataFrame
zoo_final

In [ ]:
writer = pd.ExcelWriter("EXPORT_zoo_final_con_grafico.xlsx", engine="xlsxwriter")

# Convertimos el dataframe a un XlsxWriter Excel object
zoo_final.to_excel(writer, sheet_name="AguaNecesaria")

# Parametros del workbook
#workbook  = writer.book # No se utiliza en este ejemplo
worksheet = writer.sheets["AguaNecesaria"]

rutaImagen = "./AguaNecesaria.jpg"

# Insertamos la imagen.
worksheet.insert_image("F3", rutaImagen, {'x_scale': 0.75, 'y_scale': 0.75})

# Cerramos y guardamos
writer.save()